# Classificador

**Dupla:** **Thalia Loiola | Laís Nascimento**   
   
**Turma: B**

**Introdução**

Para a realização do projeto, foi escolhido pela dupla como tema a ser analisado a marca chinesa Xiaomi, que está ganhando espaço no mercado, atualmente. Com base nos mais de 500 tweets colhidos do Twitter, esses tweets foram divididos em grupos de pessoas que são a favor dos aparelhos da Xiaomi e pessoas que são contra essa tecnologia. Nesse contexto, foi estabelecido o valor 0 para o primeiro grupo e 1 para o segundo, de forma que fossem classificados 330 tweets para treinamento do programa e 200 para teste de veracidade e eficiência do programa desenvolvido. 

Vale pontuar que a dupla utilizou-se de artificios matemáticos de probabilidade - o método de Naive Bayes - para maior precisão do programa.

**Método utilizado para análise dos tweets**

Foram considerados os tweets a favor todos as aqueles que passavam uma ideia de que o autor da frase estava satisfeito com o produto ou expressava desejo em tê-lo. Além disso, as propagandas referentes à marca da Xiaomi foram consideradas dentro do grupo de tweets a favor.

Todos aqueles que não se encaixavam nas características citadas acima foram classificados dentro do grupo de tweets não favoráveis à marca.

**Limpeza dos dados**

Bibliotecas necessárias para o projeto:

In [1]:
import pandas as pd
import numpy as np
import re
from IPython.display import display
pd.options.display.max_rows = 13

Limpeza simples:

In [2]:
import re 

def cleanup(text):
    import string
    punctuation = '[\/!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

**1. Análise dos tweets marca Xiaomi**

A análise foi baseada na classificação dos tweets, de forma binária, sendo favoráveis a marca (0), ou contra a marca (1)

In [3]:
tweets = pd.read_excel('xiaomi_1.xlsx')

In [4]:
#apagar tweets que foram repetidos ('RTs')
n_repete_tt = tweets.drop_duplicates()
n_repete_tt;

In [5]:
dict_n_rep_tt= n_repete_tt.to_dict('list')

In [6]:
dict_n_rep_tt;

In [7]:
lista_tt = dict_n_rep_tt['Treinamento']
lista_tt;

In [8]:
lista_classificacao = dict_n_rep_tt['Classificação']

In [9]:
i=0
lista_sem_caracteres = []
for i in range(len(lista_tt)):
    novo_tt = cleanup(lista_tt[i])
    lista_sem_caracteres.append(novo_tt.lower())
    i += 1

In [10]:
dict_df = {}
    
dict_df['Treinamento'] = lista_sem_caracteres
dict_df['Classificação'] = lista_classificacao

Após a limpeza dos dados, teremos o um novo dataframe para os tweets de treinamento:

In [11]:
#novo banco de dados apos limpeza simples dos tweets
novo_tweets = pd.DataFrame(data=dict_df)
novo_tweets

,Treinamento,Classificação
0,@marcusc13 @xiaomibrasil @apple @motorolabr @s...,0
1,esse cel xiaomi é excelente 😁,0
2,@hugorambo18 não temos vagas abandonou a famí...,1
3,@alysonhoffmann comprei 2 fones da xiaomi no d...,0
4,@itsramss aqui e team xiaomi ramenex junte se ...,0
5,xiaomi eh a o anti cristo não caiam nessa http...,1
...,...,...
321,fui pesquisar sobre a xiaomi e gostei sla ache...,0
322,isso demonstra que muitos equipamentos são de ...,0
323,@mumudesp se quiser eu pego meus slides e te a...,0


In [12]:
lista_sem_caracteres = dict_n_rep_tt['Treinamento']

**2. Separação dos grupos a favor e contra a marca Xiaomi:**

2.1. Tweest a favor da marca:

In [13]:
afavor_tweets= novo_tweets.loc[(novo_tweets.Classificação==0)]

In [14]:
afavor_tweets

,Treinamento,Classificação
0,@marcusc13 @xiaomibrasil @apple @motorolabr @s...,0
1,esse cel xiaomi é excelente 😁,0
3,@alysonhoffmann comprei 2 fones da xiaomi no d...,0
4,@itsramss aqui e team xiaomi ramenex junte se ...,0
6,mano nem tô acreditando que depois de indicar...,0
7,@grrvide1 me sinto uma testemunha do xiaomi ok...,0
...,...,...
321,fui pesquisar sobre a xiaomi e gostei sla ache...,0
322,isso demonstra que muitos equipamentos são de ...,0
323,@mumudesp se quiser eu pego meus slides e te a...,0


2.2 Tweets contra a marca: 

In [15]:
n_afavor_tweets= novo_tweets.loc[(novo_tweets.Classificação==1)]

In [16]:
n_afavor_tweets

,Treinamento,Classificação
2,@hugorambo18 não temos vagas abandonou a famí...,1
5,xiaomi eh a o anti cristo não caiam nessa http...,1
12,rt @treviagabriel amiga n beije este homem el...,1
16,crossfiteiro é mais chato que fã da xiaomi,1
17,vou comprar meu xiaomi se eu tiver chorando é ...,1
19,será que esses xiaomi são bons mesmo,1
...,...,...
291,@guidofraga @matosrs_ tem um vídeo do loop inf...,1
301,e um fone xiaomi tb pq esse inferno de celular...,1
303,xiaomi lovers é quase uma religião ficam tenta...,1


Se analisarmos qual a probabilidade de a opinião pública ser a favor ou contra a Xiaomi, dentre esse universo de 330 tweets que foram classificados, uma análise bem simples seria analisar quantas vezes o número 1 ocorre nesse universo e quantas vezes o 0 ocorre.

Em vista disso, teremos:

In [17]:
# Analise bem simples dos tweets
P_1 = n_afavor_tweets['Treinamento'].count()/ (n_afavor_tweets['Treinamento'].count() + afavor_tweets['Treinamento'].count())
P_0 = 1-P_1

In [18]:
P_0

0.7400611620795108

Probabilidade de gostarem do produto Xiaomi
P(0)= 0,74

In [19]:
P_1

0.2599388379204893

Probabilidade de não gostarem do produto Xiaomi
P(1)= 0,26

**3. Frequências relativas dentro de cada grupo**

3.1. Grupo 0 - Opinião a favor dos produtos da Xiaomi:

In [20]:
# Analise de quem é a favor da Xiaomi
txt_afavor=''
for t in afavor_tweets['Treinamento']:
    t=str(t)
    txt_afavor+=t

In [21]:
afavor=pd.Series(txt_afavor.split())

In [22]:
afavor

0           @marcusc13
1        @xiaomibrasil
2               @apple
3          @motorolabr
4       @samsungbrasil
5                   na
             ...      
4167      condicionado
4168                da
4169              sala
4170               com
4171               ele
4172               🤩🤩😂
Length: 4173, dtype: object

In [23]:
#frequencias absolutas
tabela_afavor = afavor.value_counts()
tabela_afavor

xiaomi                    201
o                          96
que                        87
de                         87
e                          82
um                         77
                         ... 
47mm                        1
125                         1
tenham                      1
optar                       1
hahahaha@luitticaminha      1
justificando                1
Length: 1462, dtype: int64

In [24]:
#palavras no grupo que são a favor sem que haja repetição
lista_afavor = []
for palavra in tabela_afavor.index:
    lista_afavor.append(palavra)

In [25]:
#frequencias relativas
tabela_afavor_relativa = afavor.value_counts(True)

In [26]:
tabela_afavor_relativa

xiaomi                    0.048167
o                         0.023005
que                       0.020848
de                        0.020848
e                         0.019650
um                        0.018452
                            ...   
47mm                      0.000240
125                       0.000240
tenham                    0.000240
optar                     0.000240
hahahaha@luitticaminha    0.000240
justificando              0.000240
Length: 1462, dtype: float64

In [27]:
freq_afavor_min = tabela_afavor_relativa.min()

3.2. Grupo 1 - Opinião contra os produtos da Xiaomi:

In [28]:
# Analise de quem é contra a Xiaomi
txt_contra=''
for e in n_afavor_tweets['Treinamento']:
    e=str(e)
    txt_contra+=e

In [29]:
contra=pd.Series(txt_contra.split())

In [30]:
#frequencias absolutas
tabela_n_afavor = contra.value_counts()
tabela_n_afavor

xiaomi              71
de                  25
a                   24
que                 23
um                  22
o                   19
                    ..
zttwczltcucâmera     1
conversei            1
vendo                1
dá                   1
popopo               1
conecta              1
Length: 531, dtype: int64

In [31]:
#palavras no grupo que são contra sem que haja repetição
lista_contra = []
for palavra in tabela_n_afavor.index:
    lista_contra.append(palavra)

In [32]:
#frequencias relativas
tabela_contra_relativa = contra.value_counts(True)

In [33]:
tabela_contra_relativa

xiaomi              0.067943
de                  0.023923
a                   0.022967
que                 0.022010
um                  0.021053
o                   0.018182
                      ...   
zttwczltcucâmera    0.000957
conversei           0.000957
vendo               0.000957
dá                  0.000957
popopo              0.000957
conecta             0.000957
Length: 531, dtype: float64

In [34]:
freq_contra_min = tabela_contra_relativa.min()

In [35]:
# Assuma que a soma das palavras do texto a favor e do texto contra formam o meu conjunto universo para analise 
todas_palavras = txt_afavor + txt_contra
todas_palavras;

In [36]:
palavras = pd.Series(todas_palavras.split())

In [37]:
# lista com todas as palavras contidas no universo que estamos analisando
lista_palavras_treinamento = []
for palavra in palavras:
    lista_palavras_treinamento.append(palavra)

In [38]:
# Probabilidade de cada palavra aparecer no espaço amostral (contra + a favor) que estamos trabalhando
tabela_palavras_relativa = palavras.value_counts(True)
tabela_palavras_relativa

xiaomi         0.052137
o              0.022043
de             0.021468
que            0.021085
e              0.019360
um             0.018976
                 ...   
ar             0.000192
coitado        0.000192
prova          0.000192
serei          0.000192
checkxiaomi    0.000192
7hrs           0.000192
Length: 1769, dtype: float64

In [39]:
#calculo do numero de palavras que existem em ambas as categorias de tweets
universo = palavras.count()
universo

5217

In [40]:
# Probabilidade de não gostar --> P(1)

total_contra = tabela_n_afavor.sum()
P_1 = total_contra/universo
P_1

0.2003066896683918

In [41]:
# Probabilidade de gostar --> P(0)

total_afavor = tabela_afavor.sum()
P_0 = total_afavor/universo
P_0

0.7998849913743531

**4. Fase de teste**

In [42]:
testando = pd.read_excel('teste.xlsx')

In [43]:
lista_teste=[]
for t in testando['Teste']:
    t_limpo = cleanup(t.lower())
    lista_teste.append(t_limpo)
    
lista_palavras_teste = []
for tweet in lista_teste:
    lista_palavras_teste.append(tweet.split())

In [73]:
teste_classificacao=[]

i=0
while i < len(lista_palavras_teste):
    #pega cada lista da lista de teste e compara com a lista de treinamento (conjunto universo)
    conjunto_teste = set(lista_palavras_teste[i])
    conjunto_treinamento_afavor = set(lista_afavor)
    conjunto_treinamento_contra = set(lista_contra)
    
    # queremos a interseção dos conjuntos. Ou seja, todas as palavras que aparecem em teste, devem aparecer no treinamento. 
    # Para as que não aparecerem, iremos fazer a suavização de Laplace
    inter_buscada_afavor = list(conjunto_teste.intersection(conjunto_treinamento_afavor))
    inter_buscada_contra = list(conjunto_teste.intersection(conjunto_treinamento_contra))
     
    #tudo oq tem nos tweets de teste e nao tem na interseção
    diferenca_afavor = list(conjunto_teste.difference(set(inter_buscada_afavor)))
    diferenca_contra = list(conjunto_teste.difference(set(inter_buscada_contra)))
     
    if len(diferenca_afavor)==0: #significa que todas as palavras de teste estão presentes no treinamento
        P_tweet_gosta = tabela_afavor_relativa[lista_palavras_teste[i]].prod()

    else:
        prod_afavor = ((freq_afavor_min/2)**len(diferenca_afavor))
        P_tweet_gosta = (tabela_afavor_relativa[inter_buscada_afavor].prod())*(prod_afavor)
        
    if len(diferenca_contra)==0:
        P_tweet_ngosta = tabela_contra_relativa[lista_palavras_teste[i]].prod()
    
    else:
        prod_contra = ((freq_contra_min/2)**len(diferenca_contra))
        P_tweet_ngosta = (tabela_contra_relativa[inter_buscada_contra].prod())*(prod_contra)
            
    P_gosta_tweet = P_tweet_gosta*P_0
    P_ngosta_tweet = P_tweet_ngosta*P_1 
    
    if P_gosta_tweet>P_ngosta_tweet:
        teste_classificacao.append(0)
    else: 
        teste_classificacao.append(1)
        
    i+=1
            

In [74]:
dict_teste = {}
    
dict_teste['Teste'] = lista_teste
dict_teste['Classificação'] = teste_classificacao

In [75]:
classificados = pd.DataFrame(data=dict_teste)
classificados

,Teste,Classificação
0,xiaomi é o celular que emite mais radiação só ...,1
1,kkkkkkkkkkk os fanboys de xiaomi espumando no ...,1
2,dei um tombo no meu celular ontem na plataform...,1
3,@sabrina27114057 compra um xiaomi,1
4,@babiie_ @olhardigital acabei de pegar um xiao...,1
5,olhem lá se eu usasse a marca do meu telemóvel...,1
...,...,...
194,@xiaomi_br @brunnarangel foi indicada ao emmy ...,1
195,🍃🌧️ já é quase sexta feira \n\n mi9t\n xiao...,1
196,cabo de dados e carga rápida blitzwolf 2 in 1 ...,1


In [76]:
P_0_nova = teste_classificacao.count(0)/len(teste_classificacao)
P_0_nova

0.105

In [77]:
P_1_nova = teste_classificacao.count(1)/len(teste_classificacao)
P_1_nova

0.895

## Comparação

In [63]:
comparando= pd.read_excel('pasta1.xlsx')

In [64]:
c=[]
for n in comparando['Classificação']:
    c.append(n)

In [65]:
dict_comparando = {}
    
dict_comparando['Teste'] = lista_teste
dict_comparando['Classificação'] = teste_classificacao
dict_comparando['Classificação Real']= c

In [66]:
comparacacao = pd.DataFrame(data=dict_comparando)
comparacacao

,Teste,Classificação,Classificação Real
0,xiaomi é o celular que emite mais radiação só ...,1,1
1,kkkkkkkkkkk os fanboys de xiaomi espumando no ...,1,1
2,dei um tombo no meu celular ontem na plataform...,1,0
3,@sabrina27114057 compra um xiaomi,1,0
4,@babiie_ @olhardigital acabei de pegar um xiao...,1,1
5,olhem lá se eu usasse a marca do meu telemóvel...,1,0
...,...,...,...
194,@xiaomi_br @brunnarangel foi indicada ao emmy ...,1,0
195,🍃🌧️ já é quase sexta feira \n\n mi9t\n xiao...,1,0
196,cabo de dados e carga rápida blitzwolf 2 in 1 ...,1,0


In [67]:
lista_real= comparacacao['Classificação Real'].tolist()
lista_classificador= comparacacao["Classificação"].tolist()
contagem=0

for i, elemento in enumerate(lista_real):
    elemento=int(elemento)
    if elemento == lista_classificador[i]:
        contagem+=1
        
probabilidade_de_acertos=contagem/(len(lista_real))
print("Os resultados obtidos pelo nosso programa concordam em ",probabilidade_de_acertos*100,"% das vezes com as previsões." )

Os resultados obtidos pelo nosso programa concordam em  28.999999999999996 % das vezes com as previsões.


## Conclusão

**Base de treinamento**

Segundo a base de treinamento, pode-se notar que a probabilidade de a opinião pública ser favorável à marca Xiaomi era de, aproximadamente, 80%. Portanto, a de a opinião pública ser desfavorável era de 20%.


**Base de teste**

Nota-se que de acordo com a base de teste a probabilidade de ocorrência de comentários a favor da marca era 11% e a de ocorrências negativas era de 89%. Todavia, percebe-se que esse não era o esperado com base no banco de dados de tweets que foi analisado para treinamento.

Diante disso, pode-se dizer que esse fator pode ter acontecido por uma falta de refinamento durante a análise dos tweets, uma vez que o programa desenvolvido acima não é capaz de identificar frases que possuem um viés sarcástico ou mesmo frases que possuem um duplo sentido, não podendo, assim, ser identificado qual o sentimento que o autor da frase queria trasmitir. Além disso, pode-se citar também as marcações com o "@", os emojis que não foram considerados durante o treinamento do programa e o surgimento de uma nova frequência relativa das palavras inéditas que apareceram no teste - calculada como a menor frequência relativa dentro no grupo no qual ela está sendo analisada, dividida por dois -, que implicará na diminuição da probabilidade do tweet que a contém estar nesse grupo. E, caso analise-se um tweet que tem mais palavras inéditas em um grupo do que em outro, haverá também uma considerável diferença entre as probabilidades.

Nesse contexto, é válido ressaltar que esse defeito no classificador ocorreu tendo em vista que as probabilidades condicionais de cada palavra no tweet são independentes entre sim. 

Portanto, para aperfeiçoamento desse classificador, além do treinamento de mais tweets para evitar o aparecimento frequente de palavras inéditas,tais pontos citados acima deveriam ser levados em consideração. 